In [14]:
import tensorflow as tf
import input_data
import numpy as np
mnist = input_data.read_data_sets("../data/MNIST_data/", one_hot=True)
checkpoint_dir = "/Users/kimjun/Documents/mywork/ML lec/model/"

learning_rate = 0.001
training_epochs = 15 # node count
display_step = 1    # 
batch_size = 128
test_size = 256

Extracting ../data/MNIST_data/train-images-idx3-ubyte.gz


/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/gzip.py:274: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  return self._buffer.read(size)
/Users/kimjun/Documents/mywork/ML lec/ML lec 11 - CNN/input_data.py:35: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Extracting ../data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ../data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ../data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [15]:
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [16]:
def model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    l1a = tf.nn.relu(tf.nn.conv2d(X, w,                       # l1a shape=(?, 28, 28, 32)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l1 = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1],              # l1 shape=(?, 14, 14, 32)
                        strides=[1, 2, 2, 1], padding='SAME')
    l1 = tf.nn.dropout(l1, p_keep_conv)

    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2,                     # l2a shape=(?, 14, 14, 64)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l2 = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1],              # l2 shape=(?, 7, 7, 64)
                        strides=[1, 2, 2, 1], padding='SAME')
    l2 = tf.nn.dropout(l2, p_keep_conv)

    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3,                     # l3a shape=(?, 7, 7, 128)
                        strides=[1, 1, 1, 1], padding='SAME'))
    l3 = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1],              # l3 shape=(?, 4, 4, 128)
                        strides=[1, 2, 2, 1], padding='SAME')
    l3 = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])    # reshape to (?, 2048)
    l3 = tf.nn.dropout(l3, p_keep_conv)

    l4 = tf.nn.relu(tf.matmul(l3, w4))
    l4 = tf.nn.dropout(l4, p_keep_hidden)

    pyx = tf.matmul(l4, w_o)
    return pyx

trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
trX = trX.reshape(-1, 28, 28, 1)  # 28x28x1 input img
teX = teX.reshape(-1, 28, 28, 1)  # 28x28x1 input img

X = tf.placeholder("float", [None, 28, 28, 1])
Y = tf.placeholder("float", [None, 10])

w = init_weights([3, 3, 1, 32])       # 3x3x1 conv, 32 outputs
w2 = init_weights([3, 3, 32, 64])     # 3x3x32 conv, 64 outputs
w3 = init_weights([3, 3, 64, 128])    # 3x3x32 conv, 128 outputs
w4 = init_weights([128 * 4 * 4, 625]) # FC 128 * 4 * 4 inputs, 625 outputs
w_o = init_weights([625, 10])         # FC 625 inputs, 10 outputs (labels)

p_keep_conv = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x = model(X, w, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(py_x, Y))
train_op = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op = tf.argmax(py_x, 1)

# Launch the graph in a session
with tf.Session() as sess:
    # you need to initialize all variables
    tf.initialize_all_variables().run()

    for i in range(100):
        training_batch = zip(range(0, len(trX), batch_size),
                             range(batch_size, len(trX), batch_size))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={X: trX[start:end], Y: trY[start:end],
                                          p_keep_conv: 0.8, p_keep_hidden: 0.5})

        test_indices = np.arange(len(teX)) # Get A Test Batch
        np.random.shuffle(test_indices)
        test_indices = test_indices[0:test_size]

        print(i, np.mean(np.argmax(teY[test_indices], axis=1) ==
                         sess.run(predict_op, feed_dict={X: teX[test_indices],
                                                         Y: teY[test_indices],
                                                         p_keep_conv: 1.0,
                                                         p_keep_hidden: 1.0})))

0 0.9765625
1 0.96875
2 0.98828125
3 1.0
4 0.984375
5 1.0
6 0.984375
7 0.98046875
8 0.98828125
9 0.9921875
10 1.0
11 0.99609375
12 0.98828125
13 0.9921875
14 1.0
15 0.99609375
16 0.99609375
17 0.9921875
18 0.99609375
19 0.9921875
20 1.0
21 0.9921875
22 0.98828125
23 0.984375
24 0.9921875
25 1.0
26 1.0
27 0.9921875
28 0.98828125
29 0.98046875
30 0.984375
31 1.0
32 0.98046875
33 1.0
34 1.0
35 0.9921875
36 0.98828125
37 1.0
38 0.98046875
39 0.98828125
40 0.9921875
41 0.99609375
42 0.99609375
43 0.9921875
44 0.9921875
45 0.984375
46 0.98828125
47 0.9921875
48 0.9921875
49 0.9921875
50 0.99609375
51 0.984375
52 0.99609375
53 0.99609375
54 0.9921875
55 0.99609375
56 0.9921875
57 1.0
58 0.984375
59 0.98828125
60 0.9921875
61 0.99609375
62 0.9765625
63 0.98828125
64 0.99609375
65 1.0
66 1.0
67 0.984375
68 0.98828125
69 1.0
70 0.99609375
71 0.9765625
72 0.9921875
73 0.9921875
74 0.99609375
75 1.0
76 1.0
77 0.99609375
78 0.9921875
79 0.99609375
80 1.0
81 0.9921875
82 0.9921875
83 0.9921875
84 0.